In [ ]:
pip install tsaug

In [ ]:
import numpy as np 
import pandas as pd 
import os, datetime 
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from tqdm import tqdm
import random 
import time
import pywt
import tsaug
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse 

In [ ]:
## Load data 
train_features = pd.read_csv('drive/MyDrive/movement_detection/train_features.csv')
train_labels = pd.read_csv('drive/MyDrive/movement_detection/train_labels.csv')
test_features = pd.read_csv('drive/MyDrive/movement_detection/test_features.csv')
sample_submission = pd.read_csv('drive/MyDrive/movement_detection/sample_submission.csv')

In [ ]:
X = tf.reshape(np.array(train_features.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X) 
X.shape

(3125, 600, 6)

In [ ]:
y = train_labels['label'].values
y.shape

(3125,)

In [ ]:
batch_size = 32
seq_len = 600 # temporary 
d_k = 256 
d_v = 256
n_heads = 12 
ff_dim = 512

In [ ]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')
    
        self.key = Dense(self.d_k, 
                     input_shape=input_shape, 
                     kernel_initializer='glorot_uniform', 
                     bias_initializer='glorot_uniform')
    
        self.value = Dense(self.d_v, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
    
        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    

In [ ]:
class MultiAttention(Layer):
    def __init__(self, d_k, d_v, n_heads):
        super(MultiAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.attn_heads = list()

    def build(self, input_shape):
        for n in range(self.n_heads):
            self.attn_heads.append(SingleAttention(self.d_k, self.d_v))  
    
        self.linear = Dense(input_shape[0][-1], 
                        input_shape=input_shape, 
                        kernel_initializer='glorot_uniform', 
                        bias_initializer='glorot_uniform')

    def call(self, inputs):
        attn = [self.attn_heads[i](inputs) for i in range(self.n_heads)]
        concat_attn = tf.concat(attn, axis=-1)
        multi_linear = self.linear(concat_attn)
        return multi_linear   


In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, d_k, d_v, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.ff_dim = ff_dim
        self.attn_heads = []
        self.dropout_rate = dropout 

    def build(self, input_shape):
        self.attn_multi = MultiAttention(self.d_k, self.d_v, self.n_heads)
        self.attn_dropout = Dropout(self.dropout_rate)
        self.attn_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)

        self.ff_conv1D_1 = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')
        self.ff_conv1D_2 = Conv1D(filters=input_shape[0][-1], kernel_size=1) 
        self.ff_dropout = Dropout(self.dropout_rate)
        self.ff_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)    
  
    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq) 
        attn_layer = self.attn_multi(inputs) 
        attn_layer = self.attn_dropout(attn_layer)
        attn_layer = self.attn_normalize(inputs[0] + attn_layer)
        ff_layer = self.ff_conv1D_1(attn_layer)  
        ff_layer = self.ff_conv1D_2(ff_layer)
        ff_layer = self.ff_dropout(ff_layer)
        ff_layer = self.ff_normalize(inputs[0] + ff_layer)
        return ff_layer 

    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                   'd_v': self.d_v,
                   'n_heads': self.n_heads,
                   'ff_dim': self.ff_dim,
                   'attn_heads': self.attn_heads,
                   'dropout_rate': self.dropout_rate})
        return config


In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [ ]:
def build_model(seq_len, features): 
  inputs = Input(shape=(seq_len,features))   
  bn = BatchNormalization()(inputs)    

  conv = Conv1D(256, 5, activation = 'relu', padding = 'same')(bn) 
  bn = BatchNormalization()(conv) 
  maxpool = MaxPooling1D()(bn) 
  conv = Conv1D(256, 5, activation = 'relu', padding = 'same')(maxpool) 
  bn = BatchNormalization()(conv) 
  features = MaxPooling1D()(bn) 

  pos_encoding = positional_encoding(seq_len, d_k)    
  features *= tf.math.sqrt(tf.cast(d_k, tf.float32)) # scale  
  features += pos_encoding[:, :features.shape[1], :] # add positional encoding 

  attn_layer1 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer2 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer3 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)

  x = attn_layer1((features, features, features))   
  x = attn_layer2((x, x, x)) 
  x = attn_layer3((x, x, x)) 

  bi_gru = Bidirectional(GRU(128, return_sequences = False))(x) 
  dropout = Dropout(0.25)(bi_gru) 
  dense = Dense(128, activation = 'relu')(dropout) 
  bn = BatchNormalization()(dense) 
  outputs = Dense(61, activation='softmax')(bn) 
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
kfold = StratifiedKFold(n_splits = 10, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(kfold.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1))  
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = [] 

    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(10): # add random shift 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
        for j in range(10): # add noise 
            noised = np.random.normal(0, 1, X_train[i].shape) + X_train[i] 
            X_augmented.append(noised) 
            y_augmented.append(y_train[i])   
    

    X_cropped = Crop(random.randint(300, 580), resize = 600).augment(X_train) 
    X_warped = TimeWarp(random.randint(3,20)).augment(X_train)  
    #X_reversed = Reverse().augment(X_train) 
    X_quantized = Quantize(random.randint(10,100)).augment(X_train) 
    x_drift = Drift(max_drift=(0.1,0.5)).augment(X_train)
    x_drift2 = Drift(max_drift=(0.1,0.5)).augment(X_train)

    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)    

    X_train = np.concatenate([X_train, X_augmented, X_cropped, X_warped, x_drift, x_drift2, X_quantized]) 
    y_train = np.concatenate([y_train, y_augmented, y_train, y_train, y_train, y_train, y_train])  

    print("Train shapes")
    print(X_train.shape, y_train.shape)  

    ###### feature engineering data ##### 
    print("... DFFT Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)  
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)

    ##### train model #####  
    print("... Building Model ...")
    # we have 18 features after feature engineering 
    model = build_model(600, 18) 
    print("... Training ...") 
    model_path = 'drive/MyDrive/movement_detection/kfold' + str(idx+1) + '/TRANSFORMER5_epoch_{epoch:03d}_val_{val_loss:.3f}_accuracy_{val_accuracy:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 1, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32, # hyperparameter  
              validation_data = (X_val, y_val), 
              callbacks = [learning_rate_reduction, checkpoint, early_stopping])



  2%|▏         | 45/2812 [00:00<00:06, 443.52it/s]

... Validating on fold 1 ...
... Augmenting Data ...


  1%|▏         | 948/73112 [00:00<00:07, 9471.52it/s]

Train shapes
(73112, 600, 6) (73112,)
... DFFT Feature Engineering ...


100%|██████████| 313/313 [00:00<00:00, 9472.05it/s]


...


Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.0002621440216898918.

Epoch 00012: val_loss did not improve from 0.80734
... Validating on fold 9 ...
... Augmenting Data ...


  1%|          | 897/73138 [00:00<00:08, 8963.41it/s]

Train shapes
(73138, 600, 6) (73138,)
... DFFT Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 8566.36it/s]


... Building Model ...
... Training ...
Epoch 1/200
2286/2286 [==============================] - 721s 304ms/step - loss: 2.2662 - accuracy: 0.5047 - val_loss: 1.4495 - val_accuracy: 0.6442

Epoch 00001: val_loss improved from inf to 1.44951, saving model to drive/MyDrive/movement_detection/kfold9/TRANSFORMER5_epoch_001_val_1.450_accuracy_0.644.h5
Epoch 2/200
2286/2286 [==============================] - 700s 306ms/step - loss: 1.2638 - accuracy: 0.6536 - val_loss: 1.1111 - val_accuracy: 0.7276

Epoch 00002: val_loss improved from 1.44951 to 1.11108, saving model to drive/MyDrive/movement_detection/kfold9/TRANSFORMER5_epoch_002_val_1.111_accuracy_0.728.h5
Epoch 3/200
2286/2286 [==============================] - 703s 308ms/step - loss: 0.9289 - accuracy: 0.7312 - val_loss: 1.0489 - val_accuracy: 0.7468

Epoch 00003: val_loss improved from 1.11108 to 1.04892, saving model to drive/MyDrive/movement_detection/kfold9/TRANSFORMER5_epoch_003_val_1.049_accuracy_0.747.h5
Epoch 4/200
2286/2286 [==

In [ ]:
## Make prediction
model1 = load_model('drive/MyDrive/movement_detection/kfold1/testNoise_epoch_004_val_0.905_accuracy_0.757.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model2 = load_model('drive/MyDrive/movement_detection/kfold2/testNoise_epoch_002_val_1.010_accuracy_0.725.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model3 = load_model('drive/MyDrive/movement_detection/kfold3/testNoise_epoch_004_val_0.957_accuracy_0.751.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})  
model4 = load_model('drive/MyDrive/movement_detection/kfold4/testNoise_epoch_002_val_0.994_accuracy_0.709.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model5 = load_model('drive/MyDrive/movement_detection/kfold5/testNoise_epoch_004_val_0.966_accuracy_0.748.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model6 = load_model('drive/MyDrive/movement_detection/kfold6/testNoise_epoch_004_val_0.854_accuracy_0.772.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model7 = load_model('drive/MyDrive/movement_detection/kfold7/testNoise_epoch_003_val_1.004_accuracy_0.731.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model8 = load_model('drive/MyDrive/movement_detection/kfold8/testNoise_epoch_004_val_0.859_accuracy_0.763.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model9 = load_model('drive/MyDrive/movement_detection/kfold9/testNoise_epoch_002_val_1.107_accuracy_0.734.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model10 = load_model('drive/MyDrive/movement_detection/kfold10/testNoise_epoch_004_val_0.966_accuracy_0.763.h5',
                     custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})


In [ ]:
X_test = tf.reshape(np.array(test_features.iloc[:,2:]),[-1, 600, 6])
X_test = np.asarray(X_test)

##### feature engineering for test dataset ##### 
X_test_fourier_real = [] 
X_test_fourier_imag = [] 
for i in tqdm(range(X_test.shape[0]), position = 0, leave = True):
    real_part = np.fft.fft(X_test[i]).real 
    imag_part = np.fft.fft(X_test[i]).imag 
    X_test_fourier_real.append(real_part) 
    X_test_fourier_imag.append(imag_part)
    
X_test_fourier_real = np.asarray(X_test_fourier_real) 
X_test_fourier_imag = np.asarray(X_test_fourier_imag)
X_test = np.concatenate([X_test, X_test_fourier_real, X_test_fourier_imag], axis = 2)  

print(X_test.shape)


In [ ]:
p1 = model1.predict(X_test) 
p2 = model2.predict(X_test)
p3 = model3.predict(X_test) 
p4 = model4.predict(X_test) 
p5 = model5.predict(X_test) 
p6 = model6.predict(X_test) 
p7 = model7.predict(X_test) 
p8 = model8.predict(X_test)
p9 = model9.predict(X_test)
p10 = model10.predict(X_test)


In [ ]:
p_avg = (p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10)/10.0

sample_submission.iloc[:,1:] = p_avg

sample_submission.to_csv("drive/MyDrive/movement_detection/TRANSFORMER5.csv",index=False)
